Notas de cómo hacer algunas cosas con R
=======================================

**Author:** Marcos Bujosa



<div class="ABSTRACT" id="org4668cf4">
<p>
Algunas notas de cómo hacer cosas en R.
</p>

</div>



#### Carga de algunas librerías de R que vamos a usar aquí



In [1]:
library(readr)        # para leer ficheros CSV
library(zoo)          # para leer datos con índices temporales
library(ggplot2)
library(jtools)       # para representación resultados estimación (summ)

y además fijamos los parámetros por defecto para las figuras en `png`
del notebook



In [1]:
# fijamos el tamaño de las figuras que se generan en el notebook
options(repr.plot.width = 12, repr.plot.height = 4, repr.plot.res = 200)

## De `CSV` a `data_frame`



In [1]:
datos_df <- read_csv('datos/GNPvsMelanoma.csv',show_col_types = FALSE)
head(datos_df, 3)

## De `data_frame` a `ts`



Este método permite volver al `data_frame`



In [1]:
datos_ts   <- ts(data = datos_df,
                 start = 1936,
                 end = 1972,
                 frequency = 1)
head(datos_ts, 3)

## De `CSV` a `ts` (vía `zoo`)



### Primero `zoo`



In [1]:
data <- read.zoo('datos/GNPvsMelanoma.csv', header=TRUE, index.column = 1, sep=",", FUN = as.yearmon)
class(data)
head(data, 3)
plot(data)
plot(data[,'Melanoma'])

### Luego a `ts`



In [1]:
data_ts = as.ts( data )
class(data_ts)
head(data_ts, 3)

plot(data_ts[,'GNP'])

## De dataframe a ts



In [1]:
#+BEGIN_SRC jupyter-R
tseries <- as.ts( read.zoo( datos_df ) )
head(tseries)
plot(tseries)

## De `ts` a `data_frame`



In [1]:
DF = data.frame(date = zoo::as.Date(time(tseries[,"GNP"])),
                GNP = as.matrix(tseries[,"GNP"]),
                Melanoma = as.matrix(tseries[,"Melanoma"]))
head(DF,3)

## Resumen: dos formas de pasar de datos anuales en `CSV` a `ts`



Una forma



In [1]:
time_series <- ts(data = read_csv('datos/GNPvsMelanoma.csv',show_col_types = FALSE),
                  start = 1936,
                  end = 1972,
                  frequency = 1)
head(time_series, 2)

donde `read_csv('datos/GNPvsMelanoma.csv',show_col_types = FALSE)` es un `data_frame`

Otra forma (vía `zoo`)



In [1]:
# library(zoo)
data_ts = as.ts( read.zoo('datos/GNPvsMelanoma.csv', header=TRUE, index.column = 1, sep=",", FUN = as.yearmon) )
head(data_ts, 3)

## Gráfico de dos series temporales con sendos ejes verticales



In [1]:
# Mostrando la serie GNP
p <- autoplot(as.zoo(data_ts[,'GNP']))
p <- p + geom_line(aes(y = as.zoo(data_ts[,'GNP'])), colour="blue")

# como tienen escalar distintas se requiere ajustar los datos
sf<-max(data_ts[,'GNP'])/max(data_ts[,'Melanoma'])

# Se agrega Melanoma a Y multiplicada por el factor
p <- p + geom_line(aes(y = as.zoo(data_ts[,'Melanoma'])*sf), colour="red")
p <- p + scale_y_continuous(sec.axis = sec_axis(~./sf, name = "Incidencia casos de melanoma"))
p <- p + labs(y = "GNP",
              x = "Fechas")

# Se modifican los colores de los ejes
p <- p + theme(
    axis.title.y.left=element_text(color="blue"),
    axis.text.y.left=element_text(color="blue"),
    axis.ticks.y.left = element_line(color = "blue"),
    axis.title.y.right=element_text(color="red"),
    axis.text.y.right=element_text(color="red"),
    axis.ticks.y.right = element_line(color = "red")
  )
p

### Desde el dataframe



In [1]:
# Mostrando la serie GNP
p <- ggplot(DF, aes(x = date))
p <- p + geom_line(aes(y = GNP), colour="blue")

# como tienen escalar distintas se requiere ajustar los datos
sf<-max(DF['GNP'])/max(DF['Melanoma'])

# Se agrega Melanoma a Y multiplicada por el factor
p <- p + geom_line(aes(y = Melanoma*sf), colour="red")

p <- p + scale_y_continuous(sec.axis = sec_axis(~./sf, name = "Casos de melanoma"))
p <- p + labs(y = "GNP",
              x = "Fechas")
# Se modifican los colores de los ejes
p <- p + theme(
    axis.title.y.left=element_text(color="blue"),
    axis.text.y.left=element_text(color="blue"),
    axis.ticks.y.left = element_line(color = "blue"),
    axis.title.y.right=element_text(color="red"),
    axis.text.y.right=element_text(color="red"),
    axis.ticks.y.right = element_line(color = "red")
  )
p

#### Y otra manera con `xyplot` (de `latticeExtra`)



Y ahora generamos el gráfico



In [1]:
library(latticeExtra) # alternativa para gráficos con doble eje vertical (doubleYScale)
kk <- xyplot(GNP + Melanoma ~ date, DF, type="l")
# se agrega dos ejes Y se construye cada serie por separado
obj1 <- xyplot(GNP ~ date, DF, type = "l" , lwd=2, ylab="GNP (miles de millones de $)",  xlab="Years")
obj2 <- xyplot(Melanoma ~ date, DF, type = "l", lwd=2, ylab="Casos de melanoma")
# --> se realiza la grafica con el segundo eje Y
doubleYScale(obj1, obj2, add.ylab2 = TRUE)

## Añadir nueva columna a un `ts`



In [1]:
d_GNP      = diff(datos_ts[,"GNP"])
d_Melanoma = diff(datos_ts[,"Melanoma"])

este método cambia el nombre de las primeras columnas



In [1]:
serie = ts.union(data_ts, d_GNP, d_Melanoma)
head(serie,3)

Este método es mejor, pero es pesado



In [1]:
serie = ts.union(GNP = data_ts[,'GNP'], Melanoma = data_ts[,'Melanoma'], d_GNP, d_Melanoma)
head(serie,3)

In [1]:
DF.diferencias = data.frame(date = zoo::as.Date(time(d_GNP)),
                            d_GNP = as.matrix(d_GNP),
                            d_Melanoma = as.matrix(d_Melanoma))
head(DF.diferencias, 2)

## Otros



In [1]:
mal_modelo <- lm(d_GNP ~ d_Melanoma)
summ( mal_modelo )

In [1]:
plot(as.ts(resid(mal_modelo))) 
abline(0,0)